In [1]:
import numpy as np
import pandas as pd
from scipy.stats import truncnorm, poisson
from scipy.optimize import minimize

# Usando um parâmetro por jogador

In [2]:
def generate_games():
    games = [[chr(i + 65) for i in range(20)] for j in range(20)]
    for i in range(20):
        games[i].pop(i)
        
    return games

def generate_players(temps = 15):
    players = list(np.random.normal(70, 10, 220))
    players = [round(player, 1) if player < 100 else 100 for player in players]
    clubs = {}
    for year in range(temps):
        line_up = {}
        if type(players) != list:
            np.random.shuffle(list(players.flat))

        players = np.reshape(players, (20, 11))
        for club in range(20):
            line_up[chr(65 + club)] = players[club]

        clubs[year] = line_up
        
    return clubs

def find_median(array):
    arraycopy = array.copy()
    arraycopy.sort()
    if len(arraycopy) % 2 == 0:
        return (arraycopy[len(arraycopy) // 2 - 1] + arraycopy[len(arraycopy) // 2])/2
    else:
        return arraycopy[len(arraycopy) // 2]
    
def find_forces(clubs, year = 0):
    atk_forces = []
    def_forces = []
    for club in clubs[year]:
        atk_forces.append(np.sum(clubs[year][club]))
        def_forces.append(np.log(np.prod(clubs[year][club])))
            
    atk_med = find_median(atk_forces)
    def_med = find_median(def_forces)
    
    for i in range(len(atk_forces)):
        atk_forces[i] = atk_forces[i] / atk_med * 2
        def_forces[i] /= def_med
            
    return atk_forces, def_forces

def model(club1, club2, sims = 10000):
    '''
    receives 2 clubs (forces) and returns the most likely result, probability of each result and probability for the game
    '''
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson.rvs(club1[0] - club2[1])
        goals2 = poisson.rvs(club2[0] - club1[1])
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [3]:
def championship(clubs, sims = 10000, year = 0):
    atk_forces, def_forces = find_forces(clubs, year = year)
    games = generate_games()
    results = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            club1 = (atk_forces[home], def_forces[home])
            club2 = (atk_forces[away], def_forces[away])
            _, result, _ = model(club1, club2, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results

In [4]:
clubs = generate_players(temps = 1)
atk_forces, def_forces = find_forces(clubs)

# exemplo
club1 = (atk_forces[ord('A') - 65], def_forces[ord('A') - 65])
club2 = (atk_forces[ord('B') - 65], def_forces[ord('B') - 65])
results, result, probs = model(club1, club2, sims = 10000)

print('Forças do clube 1:', club1)
print('Forças do clube 2:', club2)
print()
print('Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):')
print(results)
print()
print('O resultado mais provável é {} x {}.'.format(result[0], result[1]))
print('Além disso, a probabilidade do mandante é {}, enquanto a do visitante é de {} e a de empate é {}.'.format(probs[0], probs[2], probs[1]))

Forças do clube 1: (2.088138276817522, 1.0098780578662614)
Forças do clube 2: (2.089985486211901, 1.0109160508323136)

Quantidade de vezes que o placar foi i x j (linha 5 e coluna 5 representam 5 ou mais gols):
[[1162 1306  618  254   61   13]
 [1202 1360  733  255   65   20]
 [ 640  728  405  152   43    8]
 [ 249  295  136   46   17    4]
 [  62   59   31   17    3    3]
 [  22   18    6    5    2    0]]

O resultado mais provável é 1 x 1.
Além disso, a probabilidade do mandante é 0.3472, enquanto a do visitante é de 0.3552 e a de empate é 0.2976.


In [5]:
df, results = championship(clubs, sims = 100)
df

,points,games,wins,draws,defeats,goals for,goals against,goal difference
A,69,38,18,15,5,27,12,15
H,64,38,15,19,4,25,14,11
R,59,38,14,17,7,20,13,7
S,57,38,13,18,7,25,19,6
J,57,38,13,18,7,23,17,6
I,56,38,13,17,8,24,19,5
O,48,38,9,21,8,21,20,1
Q,48,38,8,24,6,26,24,2
P,47,38,11,14,13,17,19,-2
B,47,38,7,26,5,24,22,2


In [6]:
results

[['A', 1, 1, 'B'],
 ['A', 1, 1, 'C'],
 ['A', 1, 0, 'D'],
 ['A', 1, 0, 'E'],
 ['A', 1, 0, 'F'],
 ['A', 0, 0, 'G'],
 ['A', 0, 0, 'H'],
 ['A', 1, 0, 'I'],
 ['A', 1, 0, 'J'],
 ['A', 0, 0, 'K'],
 ['A', 0, 0, 'L'],
 ['A', 2, 0, 'M'],
 ['A', 1, 0, 'N'],
 ['A', 0, 1, 'O'],
 ['A', 1, 0, 'P'],
 ['A', 0, 1, 'Q'],
 ['A', 0, 1, 'R'],
 ['A', 0, 1, 'S'],
 ['A', 1, 0, 'T'],
 ['B', 1, 1, 'A'],
 ['B', 1, 0, 'C'],
 ['B', 1, 0, 'D'],
 ['B', 1, 1, 'E'],
 ['B', 0, 0, 'F'],
 ['B', 1, 1, 'G'],
 ['B', 0, 0, 'H'],
 ['B', 0, 0, 'I'],
 ['B', 1, 1, 'J'],
 ['B', 1, 0, 'K'],
 ['B', 0, 0, 'L'],
 ['B', 1, 0, 'M'],
 ['B', 0, 0, 'N'],
 ['B', 1, 1, 'O'],
 ['B', 1, 0, 'P'],
 ['B', 1, 1, 'Q'],
 ['B', 0, 1, 'R'],
 ['B', 1, 0, 'S'],
 ['B', 0, 0, 'T'],
 ['C', 0, 1, 'A'],
 ['C', 0, 0, 'B'],
 ['C', 1, 0, 'D'],
 ['C', 0, 0, 'E'],
 ['C', 1, 1, 'F'],
 ['C', 0, 0, 'G'],
 ['C', 0, 0, 'H'],
 ['C', 1, 1, 'I'],
 ['C', 0, 0, 'J'],
 ['C', 0, 1, 'K'],
 ['C', 0, 1, 'L'],
 ['C', 1, 0, 'M'],
 ['C', 1, 1, 'N'],
 ['C', 0, 0, 'O'],
 ['C', 0, 1,

Perceba que fazendo

$força~de~ataque~do~time = \sum força~dos~jogadores \text{ e } força~de~defesa~do~time = \sum \log{força~dos~jogadores},$

um time com maior força de ataque terá maior força de defesa, o que não é compatível com a realidade.

# Usando dois parâmetros por jogador

Cada jogador tem dois parâmetros: ataque e defesa

In [7]:
def generate_players(clubs = 20, players_per_club = 23):
    '''
    creates players
    '''
    n_players = clubs * players_per_club
    players_atk = truncnorm.rvs(0, 10, loc = 2, scale = 0.5, size = n_players)
    players_def = truncnorm.rvs(0, 10, loc = 1, scale = 0.5, size = n_players)
    players = {}
    for i in range(n_players):
        players['P{:04d}'.format(i)] = [round(players_atk[i], 8) if players_atk[i] < 100 else 100,
                                        round(players_def[i], 8) if players_def[i] < 100 else 100]
        
    return players

def change_clubs(clubs, changes):
    '''
    change clubs
    '''
    clubs_f = {}
    changing = []
    change = 0
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        np.random.shuffle(club_players)
        for j in range(changes[i]):
            changing.append(club_players[j])

    np.random.shuffle(changing)
    for i in range(n_clubs):
        club_players = clubs[chr(i + 65)].copy()
        ind = []
        for player in club_players:
            if player in changing:
                ind.append(club_players.index(player))

        for j in range(len(ind)):
            club_players[ind[j]] = changing[change]
            change += 1

        clubs_f[chr(i + 65)] = club_players

    return clubs_f

def generate_clubs(players, years = 15, n_clubs = 20, players_per_club = 23):
    '''
    receives players and allocates them to clubs
    '''
    clubs = {}
    all_players = list(players)
    for year in range(years):
        line_up = {chr(65 + club) : {} for club in range(n_clubs)}
        clubs[year] = line_up
        for club in range(n_clubs):
            if year == 0:
                clubs[year][chr(65 + club)] = all_players[players_per_club * club:players_per_club * (club + 1)]

            else:
                changes = poisson.rvs(players_per_club/4, size = n_clubs)
                for i in range(n_clubs):
                    if changes[i] > players_per_club:
                        changes[i] = players_per_club

                    clubs[year] = change_clubs(clubs[year - 1], changes)
    return clubs

In [8]:
years = 15
n_clubs = 20
players_per_club = 23
players = generate_players()
clubs = generate_clubs(players)

In [9]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

def model(club1, club2, players, sims = 10000):
    '''
    receives 2 clubs (line_up) and returns the most likely result, probability of each result and probability for the game
    '''
    atk1, def1 = find_forces(club1, players)
    atk2, def2 = find_forces(club2, players)
    results = np.zeros((6, 6), dtype = int)
    probs = [0, 0, 0]
    for i in range(sims):
        goals1 = poisson.rvs(atk1 - def2)
        goals2 = poisson.rvs(atk2 - def1)
        if goals1 > 5:
            goals1 = 5
        if goals2 > 5:
            goals2 = 5
        
        results[goals1, goals2] += 1
        
        if goals1 > goals2:
            probs[0] += 1
        elif goals1 == goals2:
            probs[1] += 1
        else:
            probs[2] += 1
    
    for i in range(3):
        probs[i] /= sims
        
    result = [0, 0] # Home - away
    for i in range(len(results)):
        for j in range(len(results[i])):
            if results[i, j] > results[result[0], result[1]]:
                result = [i, j]
            elif results[i, j] == results[result[0], result[1]]:
                # rare cases
                if np.random.random() > 0.5:
                    result = [i, j]
    
    return results, result, probs

In [10]:
def championship(clubs, players, sims = 10000, year = 0):
    games = generate_games()
    results = []
    line_up = []
    table = np.zeros((20, 8), dtype = int)
    # lines = clubs
    # columns = {points, games, wins, draws, defeats, goals for, goals against, goal difference}
    
    for i in range(len(games)):
        home = i
        for away in games[i]:
            away = ord(away) - 65
            
            club1 = clubs[year][chr(home + 65)].copy()
            np.random.shuffle(club1)
            home_line_up = club1[:11]
            
            club2 = clubs[year][chr(away + 65)].copy()
            np.random.shuffle(club2)
            away_line_up = club2[:11]
            
            _, result, _ = model(home_line_up, away_line_up, players, sims = sims)
            results.append([chr(home + 65), result[0], result[1], chr(away + 65)])
            line_up.append([home_line_up, away_line_up])
            
            # games
            table[home, 1] += 1
            table[away, 1] += 1
            
            # goals
            table[home, 5] += result[0]
            table[away, 5] += result[1]
            table[home, 6] += result[1]
            table[away, 6] += result[0]
            table[home, 7] += result[0] - result[1]
            table[away, 7] += result[1] - result[0]
            
            # results
            if result[0] > result[1]:
                table[home, 2] += 1
                table[home, 0] += 3
                table[away, 4] += 1
            elif result[0] == result[1]:
                table[home, 3] += 1
                table[away, 3] += 1
                table[home, 0] += 1
                table[away, 0] += 1
            else:
                table[home, 4] += 1
                table[away, 2] += 1
                table[away, 0] += 3
    
    columns_names = ['points', 'games', 'wins', 'draws', 'defeats', 'goals for', 'goals against', 'goal difference']
    
    table = pd.DataFrame(table,
                        index = [chr(i + 65) for i in range(20)],
                        columns = columns_names)
    table.sort_values(['points',  'wins', 'goal difference', 'goals for'],
                      axis = 0,
                      ascending = False,
                      inplace = True)
    
    return table, results, line_up

In [11]:
table, results, line_up = championship(clubs, players, sims = 100, year = 0)

In [12]:
table

,points,games,wins,draws,defeats,goals for,goals against,goal difference
L,66,38,18,12,8,69,54,15
B,62,38,15,17,6,77,68,9
G,58,38,17,7,14,71,65,6
H,56,38,14,14,10,70,66,4
C,55,38,13,16,9,72,70,2
O,55,38,12,19,7,66,57,9
Q,54,38,14,12,12,71,66,5
P,54,38,14,12,12,67,63,4
A,50,38,12,14,12,70,66,4
I,50,38,11,17,10,66,67,-1


In [13]:
results

[['A', 1, 4, 'B'],
 ['A', 1, 1, 'C'],
 ['A', 2, 2, 'D'],
 ['A', 2, 2, 'E'],
 ['A', 2, 1, 'F'],
 ['A', 1, 1, 'G'],
 ['A', 2, 2, 'H'],
 ['A', 3, 1, 'I'],
 ['A', 1, 2, 'J'],
 ['A', 2, 1, 'K'],
 ['A', 1, 1, 'L'],
 ['A', 1, 1, 'M'],
 ['A', 5, 1, 'N'],
 ['A', 2, 2, 'O'],
 ['A', 1, 3, 'P'],
 ['A', 3, 2, 'Q'],
 ['A', 1, 2, 'R'],
 ['A', 2, 1, 'S'],
 ['A', 2, 2, 'T'],
 ['B', 3, 2, 'A'],
 ['B', 2, 2, 'C'],
 ['B', 1, 1, 'D'],
 ['B', 2, 2, 'E'],
 ['B', 2, 1, 'F'],
 ['B', 2, 2, 'G'],
 ['B', 3, 2, 'H'],
 ['B', 2, 2, 'I'],
 ['B', 2, 1, 'J'],
 ['B', 1, 1, 'K'],
 ['B', 1, 1, 'L'],
 ['B', 1, 3, 'M'],
 ['B', 2, 2, 'N'],
 ['B', 2, 2, 'O'],
 ['B', 1, 1, 'P'],
 ['B', 2, 1, 'Q'],
 ['B', 4, 2, 'R'],
 ['B', 2, 2, 'S'],
 ['B', 1, 3, 'T'],
 ['C', 2, 1, 'A'],
 ['C', 2, 2, 'B'],
 ['C', 2, 2, 'D'],
 ['C', 2, 2, 'E'],
 ['C', 2, 1, 'F'],
 ['C', 2, 1, 'G'],
 ['C', 1, 3, 'H'],
 ['C', 2, 1, 'I'],
 ['C', 2, 1, 'J'],
 ['C', 1, 2, 'K'],
 ['C', 1, 1, 'L'],
 ['C', 2, 1, 'M'],
 ['C', 2, 2, 'N'],
 ['C', 2, 2, 'O'],
 ['C', 2, 2,

In [14]:
line_up

[[['P0010',
   'P0005',
   'P0008',
   'P0007',
   'P0012',
   'P0009',
   'P0021',
   'P0006',
   'P0000',
   'P0011',
   'P0022'],
  ['P0034',
   'P0028',
   'P0036',
   'P0029',
   'P0026',
   'P0030',
   'P0027',
   'P0041',
   'P0037',
   'P0023',
   'P0040']],
 [['P0018',
   'P0008',
   'P0003',
   'P0015',
   'P0009',
   'P0010',
   'P0001',
   'P0007',
   'P0014',
   'P0022',
   'P0000'],
  ['P0047',
   'P0066',
   'P0050',
   'P0059',
   'P0063',
   'P0052',
   'P0057',
   'P0056',
   'P0061',
   'P0049',
   'P0054']],
 [['P0003',
   'P0016',
   'P0020',
   'P0015',
   'P0004',
   'P0011',
   'P0008',
   'P0009',
   'P0012',
   'P0018',
   'P0002'],
  ['P0073',
   'P0081',
   'P0071',
   'P0090',
   'P0080',
   'P0069',
   'P0076',
   'P0089',
   'P0084',
   'P0078',
   'P0070']],
 [['P0005',
   'P0009',
   'P0021',
   'P0022',
   'P0017',
   'P0008',
   'P0010',
   'P0013',
   'P0007',
   'P0003',
   'P0015'],
  ['P0094',
   'P0113',
   'P0102',
   'P0105',
   'P0104',
   'P0

# Engenharia reversa

In [15]:
def find_forces(line_up, players):
    atk_force = 0
    def_force = 0
    for player in line_up:
        atk_force += 1.5*players[player][0]
        def_force += players[player][1]
    
    return atk_force/11, def_force/11

# def find_players(players_er, codes):
#     players = {}
#     for code in codes:
#         if code not in players:
#             players[code] = players_er[codes.index(code)]
    
# #     for player in players_er:
# #         if player[0] not in players:
# #             players[player[0]] = players_er[players_er.index(player)][1:]
    
#     return players

def find_players(players_er, codes):
    players = {}
    for code in codes:
        if code not in players:
            players[code] = [players_er[2 * codes.index(code)], players_er[2 * codes.index(code) + 1]]
    
    return players

def likelihood(players_er, codes, results, line_up):
    '''
    players : parameter list
    results : list
    line_up : list
    '''
    
    players_er = find_players(players_er, codes)
    loglikelihood = 0
    for game in line_up:               
        ind = line_up.index(game)
        atk1, def1 = find_forces(game[0], players_er)
        atk2, def2 = find_forces(game[1], players_er)
        
        loglikelihood += poisson.logpmf(results[ind][1], atk1 - def2)
        loglikelihood += poisson.logpmf(results[ind][2], atk2 - def1)
    
    return -loglikelihood

In [16]:
players_er = {}
codes = []
for game in line_up:
    for player in game[0]:
        if player not in players_er:
            players_er[player] = [truncnorm.rvs(0, 10, loc = 3, scale = 1), truncnorm.rvs(0, 10, loc = 1, scale = 1)]
            codes.append(player)
    for player in game[1]:
        if player not in players_er:
            players_er[player] = [truncnorm.rvs(0, 10, loc = 1, scale = 1), truncnorm.rvs(0, 10, loc = 1, scale = 1)]
            codes.append(player)
            
# players_er = [players_er[player] for player in players_er]
aux = players_er.copy()
players_er = []
for i in aux:
    players_er.append(aux[i][0])
    players_er.append(aux[i][1])

In [17]:
likelihood(players_er, codes, results, line_up)

1336.0928796050391

In [18]:
result = minimize(likelihood, players_er, args = (codes, results, line_up))

In [19]:
result

      fun: nan
 hess_inv: array([[ 1.00028694e+00,  1.24896497e-04,  3.58176260e-04, ...,
         4.80523472e-05, -2.12411571e-04,  3.16591180e-05],
       [ 1.24896497e-04,  9.99484355e-01,  1.55662627e-04, ...,
        -1.52576012e-04,  1.67788555e-04, -4.38034080e-04],
       [ 3.58176260e-04,  1.55662627e-04,  1.00044710e+00, ...,
         5.99086285e-05, -2.65035897e-04,  3.93276459e-05],
       ...,
       [ 4.80523472e-05, -1.52576012e-04,  5.99086285e-05, ...,
         9.99955242e-01,  4.36385117e-05, -1.32215582e-04],
       [-2.12411571e-04,  1.67788555e-04, -2.65035897e-04, ...,
         4.36385117e-05,  1.00003842e+00,  1.82845868e-04],
       [ 3.16591180e-05, -4.38034080e-04,  3.93276459e-05, ...,
        -1.32215582e-04,  1.82845868e-04,  9.99645365e-01]])
      jac: array([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
       nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 

In [20]:
result.nit

2